In [1]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [2]:
train= pd.read_csv('data.csv')
test_x = pd.read_csv('X_test.csv')
submit= pd.read_csv('submission.csv')
train
oversampler = SMOTE(random_state=42)
undersampler =NearMiss()
print(train.corr())

                          Pregnancies   Glucose  BloodPressure  SkinThickness  \
Pregnancies                  1.000000  0.138310       0.171798      -0.074245   
Glucose                      0.138310  1.000000       0.167664       0.019384   
BloodPressure                0.171798  0.167664       1.000000       0.154801   
SkinThickness               -0.074245  0.019384       0.154801       1.000000   
Insulin                     -0.061448  0.339681       0.052759       0.422295   
BMI                          0.030913  0.203764       0.244063       0.388597   
DiabetesPedigreeFunction    -0.022582  0.168652       0.035847       0.178307   
Age                          0.544940  0.267994       0.256916      -0.158532   
Diabetes                     0.196957  0.478535       0.112334       0.057206   

                           Insulin       BMI  DiabetesPedigreeFunction  \
Pregnancies              -0.061448  0.030913                 -0.022582   
Glucose                   0.339681  0.203

In [3]:
train_y = train['Diabetes']
train_x = train.drop(['Diabetes','SkinThickness'], axis=1)
test_x = test_x.drop(['SkinThickness'], axis=1)
train_x.describe()

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age
count,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000
mean,3.726257,120.849162,69.685289,79.836127,31.975605,0.469920,33.074488
std,3.262965,32.339523,18.094374,115.196730,7.624495,0.342087,11.685319
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,64.000000,0.000000,26.800000,0.241000,24.000000
50%,3.000000,117.000000,72.000000,36.000000,32.000000,0.374000,29.000000
75%,6.000000,139.000000,80.000000,129.000000,36.500000,0.612000,41.000000
max,17.000000,199.000000,122.000000,846.000000,67.100000,2.420000,81.000000


sc=StandardScaler()
train_x['Insulin']= sc.fit_transform(train_x.loc[:,train_x.columns=='Insulin'])
train_x['DiabetesPedigreeFunction']= sc.fit_transform(train_x.loc[:,train_x.columns=='DiabetesPedigreeFunction'])
test_x['Insulin']= sc.fit_transform(test_x.loc[:,test_x.columns=='Insulin'])
test_x['DiabetesPedigreeFunction']= sc.fit_transform(test_x.loc[:,test_x.columns=='DiabetesPedigreeFunction'])
test_x

In [4]:
mmsc=MinMaxScaler()
train_x= mmsc.fit_transform(train_x, train_y)
test_x = mmsc.transform(test_x)
train_x

array([[0.05882353, 0.47738693, 0.49180328, ..., 0.3561848 , 0.07771136,
        0.01666667],
       [0.29411765, 0.52763819, 0.59016393, ..., 0.54992548, 0.03458582,
        0.11666667],
       [0.        , 0.67839196, 0.55737705, ..., 0.63040238, 0.12254483,
        0.05      ],
       ...,
       [0.58823529, 0.50753769, 0.70491803, ..., 0.67958271, 0.45175064,
        0.28333333],
       [0.        , 0.70854271, 0.        , ..., 0.6318927 , 0.05422716,
        0.13333333],
       [0.        , 0.6281407 , 0.78688525, ..., 0.33532042, 0.07856533,
        0.        ]])

In [5]:
train_re_x, train_re_y = oversampler.fit_resample(train_x, train_y)
train_us_x, train_us_y = undersampler.fit_resample(train_x, train_y)
print(np.shape(train_y))
print(np.shape(train_re_y))
print(train_re_y.describe())
train_y.describe()

(537,)
(698,)
count    698.000000
mean       0.500000
std        0.500359
min        0.000000
25%        0.000000
50%        0.500000
75%        1.000000
max        1.000000
Name: Diabetes, dtype: float64


count    537.000000
mean       0.350093
std        0.477444
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Diabetes, dtype: float64

In [6]:
model=LinearDiscriminantAnalysis()
model2=QuadraticDiscriminantAnalysis()
model.fit(train_x,train_y)
print(model.score(train_x,train_y))
model2.fit(train_x,train_y)
print(model2.score(train_x,train_y))


model.fit(train_us_x,train_us_y)
print(model.score(train_us_x,train_us_y))
model2.fit(train_us_x,train_us_y)
print(model2.score(train_us_x,train_us_y))

model.fit(train_re_x,train_re_y)
print(model.score(train_re_x,train_re_y))
model2.fit(train_re_x, train_re_y)
model2.score(train_re_x,train_re_y)

0.7802607076350093
0.7653631284916201
0.7898936170212766
0.7872340425531915
0.7621776504297995


0.7392550143266475

In [7]:
model.fit(train_x,train_y)
print(model.score(train_x,train_y))
model2.fit(train_x,train_y)
print(model2.score(train_x,train_y))

0.7802607076350093
0.7653631284916201


In [8]:
pred_y=model.predict(test_x)
pred_y

array([0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int64)

In [9]:
submit['Diabetes']=pred_y
submit.to_csv('submission.csv', index=False)